When the action is not accepted, we have to sleep the production and wait till an action is accepted. This is because when an action is rejected, we cannot go ahead with the process, that would mean that the next item in the machine.queue would get processed. 

Aim: 

If action accepted = Wake up and move forward

If action is rejected = Sleep

I have made it work. Check out why it is working

In [1]:
from random import seed, randint
seed(23)
import simpy

## Example

In [39]:
class EV:
    def __init__(self, env):
        self.env = env
        # Driving process
        self.drive_proc = env.process(self.drive(env))
        
        # Battery control process
        self.bat_ctrl_proc = env.process(self.bat_ctrl(env))
        
        # Event - Reactivate battery control
        self.bat_ctrl_reactivate = env.event()

    def drive(self, env):
        while True:
            # Drive for 20-40 min
            #yield env.timeout(randint(20, 40))

            # Reactivate battery control
            print('Start parking at', env.now)
            self.bat_ctrl_reactivate.succeed()  # "reactivate"
            self.bat_ctrl_reactivate = env.event()
            
            # Park for 1–6 hours
            yield env.timeout(randint(60, 360))
            print('Stop parking at', env.now)

    def bat_ctrl(self, env):
        while True:
            print('Bat. ctrl. passivating at', env.now)
            yield self.bat_ctrl_reactivate  # "passivate"
            print('Bat. ctrl. reactivated at', env.now)

            # Intelligent charging behavior here …
            yield env.timeout(randint(30, 90))
            
env = simpy.Environment()
ev = EV(env)
env.run(until=200)

Start parking at 0
Bat. ctrl. passivating at 0


## My take

In [92]:
class EV:
    def __init__(self, env):
        self.env = env
        self.action_taken = randint(0,1)
        
        # Defining processes
        # Sleep process
        print("Putting machine to sleep")
        self.sleep_proc = env.process(self.sleep(env))
        
        # Production process
        self.continue_production_proc = env.process(self.continueProduction(env))

        # Defining Events        
        self.action_accepted_event = env.event()

    def continueProduction(self, env):
        while True:
            print('Choosing an action')
            self.action_taken = randint(0,1)
            
            if self.action_taken == 0: # if action accepted
                print("Action is accepted ")
                self.action_accepted_event.succeed()  # "reactivate"
                self.action_accepted_event = env.event()
                
                print('Machine reactivated at ', env.now)
                
                # Production for 2 minutes
                yield env.timeout(2)
                print('Production complete at ', env.now)
                
                # Sleep again after production
                self.sleep_proc = env.process(self.sleep(env))
                print("Putting machine to sleep at ", env.now)
                
            else:
                
                print('Action is rejected')
                
    def sleep(self, env): # sleep and production
        yield self.action_accepted_event  # "passivate"
            
            
            
            

In [93]:
env = simpy.Environment()
ev = EV(env)
env.run(until=20)

Putting machine to sleep
Choosing an action
Action is accepted 
Machine reactivated at  0
Production complete at  2
Putting machine to sleep at  2
Choosing an action
Action is rejected
Choosing an action
Action is accepted 
Machine reactivated at  2
Production complete at  4
Putting machine to sleep at  4
Choosing an action
Action is rejected
Choosing an action
Action is rejected
Choosing an action
Action is accepted 
Machine reactivated at  4
Production complete at  6
Putting machine to sleep at  6
Choosing an action
Action is accepted 
Machine reactivated at  6
Production complete at  8
Putting machine to sleep at  8
Choosing an action
Action is accepted 
Machine reactivated at  8
Production complete at  10
Putting machine to sleep at  10
Choosing an action
Action is rejected
Choosing an action
Action is rejected
Choosing an action
Action is accepted 
Machine reactivated at  10
Production complete at  12
Putting machine to sleep at  12
Choosing an action
Action is accepted 
Machine r